In [1]:
import sys

sys.path.append("../")

import cv2 as cv
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
import seaborn as sns
from scipy import stats
import os
import re
import pickle
import imutils
from scipy.signal import find_peaks, savgol_filter
from pathlib import Path
import glob
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from utils import init_condition, select_roi_interactive, load_significant_frames
from utils import noise_detection,  cfar_fast
from utils import find_periods

In [2]:
folders = ["F:\\2p_hsc\\20241017\\hsa-sizetuning\\F5",
          "F:\\2p_hsc\\20241017\\hsa-sizetuning\\F6",
           "F:\\2p_hsc\\20241022\\hsa-sizetuning\\F1",
           "F:\\2p_hsc\\20241022\\hsa-sizetuning\\F2",
           "F:\\2p_hsc\\20241024\\hsa-sizetuning\\F1",
           "F:\\2p_hsc\\20241024\\hsa-sizetuning\\F2",
           "F:\\2p_hsc\\20241025\\hsa-sizetuning\\F1",
           "F:\\2p_hsc\\20241025\\hsa-sizetuning\\F2",
           "F:\\2p_hsc\\20241029\\hsa-sizetuning\\F1",
           "F:\\2p_hsc\\20241029\\hsa-sizetuning\\F2"
          ]
folders = [str(Path(folder).resolve()) for folder in folders]

for fish in folders:
    buffer = fish.split("\\")
    fish_id = f"{buffer[-3]}-{buffer[-1]}"

    avis = glob.glob(f"{fish}/**.mp4")
    avis = [str(Path(avi).resolve()) for avi in avis]
    
    for avi in avis:
        set_no = avi.split("\\")[-1].split("-")[1]
        
        save_dir = f"D:\\swimming_bladder_data\\2p_eyefix\\{fish_id}\\{set_no}\\"
        Path(save_dir).mkdir(parents=True, exist_ok=True)
        
        if Path(f"{save_dir}/cord.txt").exists():
           with open(f"{save_dir}/cord.txt", "r") as f:
               x_cord = int(f.readline().split()[-1])
               y_cord = int(f.readline().split()[-1])
        else:
            import winsound
            frequency = 2500  # Set Frequency To 2500 Hertz
            duration = 500  # Set Duration To 1000 ms == 1 second
            winsound.Beep(frequency, duration)
            x_cord, y_cord = select_roi_interactive(avi)
            with open(f"{save_dir}/cord.txt", "w") as f:
                f.write("x\t" + str(x_cord))
                f.write("\ny\t" + str(y_cord))
                
        capture, num_frames, hsv = init_condition(avi, x_cord, y_cord)
        res = list(range(0, num_frames))
        if Path(f"{save_dir}/y_shift.npy").exists():
            continue
        else:
            vertical_motion = []
        
            capture.set(cv.CAP_PROP_POS_FRAMES, 0)
            ret, prev_frame = capture.read()
            prev_frame = prev_frame[y_cord:y_cord+50, x_cord:x_cord+50]
            prev_gray = cv.cvtColor(prev_frame, cv.COLOR_BGR2GRAY)
            
            for frame in tqdm(range(1, num_frames), desc=f"Optic Flow for {fish_id} {set_no}"):
                ret, curr_frame = capture.read()
                if not ret:
                    break
                    
                curr_frame = curr_frame[y_cord:y_cord+50, x_cord:x_cord+50]
                curr_gray = cv.cvtColor(curr_frame, cv.COLOR_BGR2GRAY)
                
                # Calculate dense optical flow
                flow = cv.calcOpticalFlowFarneback(
                    prev_gray, curr_gray, 
                    None, 0.5, 3, 15, 3, 5, 1.2, 0
                )
                
                # Focus on vertical motion (y-axis component)
                vertical_flow = flow[..., 1]
                avg_vertical = np.mean(vertical_flow)
                vertical_motion.append(avg_vertical)
                
                prev_gray = curr_gray
            vertical_motion = -np.array(vertical_motion) ###### make positive to upward motion 
            np.save(f"{save_dir}/y_shift.npy",vertical_motion)